# Prediction with LSTM model (v.4.4.6)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [6]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-29 12:49:53.668297


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed


print(f"imported library. ({datetime.now()})")

imported library. (2025-05-29 12:50:03.123889)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd4.csv'
    
mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 3

write_to_db = True


finished to set environemnt. (2025-05-29 12:50:04.004477)


In [3]:
import tensorflow as tf
print(f'{tf.version}')

<module 'tensorflow._api.v2.version' from '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [3]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }
#print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-05-29 12:50:06.647888


In [4]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_args = {14: {"model":{"learning_rate": learning_rate[0],
                           "dropout": dropout[0],
                           "steps": steps[0], # last output elements count
                           "metrics": metrics[1],
                           "last_lstm_return_sequences": last_lstm_return_sequences[0],
                           "lstm_units": lstm_units[2],
                           "dense_units": dense_units[2],
                           "output_dense_activation": output_dense_activation[3],
                           "loss": loss[0],
                           "return_state": False,
                           "rand_seed": rand_seed[0] # rand_seed[0]
                           },
                 "train": {"epochs": epochs[2],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 },
             24: {"model":{"learning_rate": learning_rate[0],
                           "dropout": dropout[0],
                           "steps": steps[0], # last output elements count
                           "metrics": metrics[1],
                           "last_lstm_return_sequences": last_lstm_return_sequences[0],
                           "lstm_units": lstm_units[2],
                           "dense_units": dense_units[2],
                           "output_dense_activation": output_dense_activation[3],
                           "loss": loss[0],
                           "return_state": False,
                           "rand_seed": rand_seed[0] # rand_seed[0]
                           },
                 "train": {"epochs": epochs[0],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 },
             }


print(f'completed to set env for all models. {datetime.now()}')

completed to set env for all models. 2025-05-29 12:50:09.763786


In [5]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 53
model_version_end = 54
args = {'test_id': 'P4_4_5_1_2',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 5,
        'round_limit': 40,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args, from_version=model_version_from, to_version=model_version_end, lstm_args=lstm_args)


Test Start: 53~54
CPU 시간: 4.155471 초
메모리 사용량: 616512 KB
********************
working model_version = 53  2025-05-29 12:50:11.753620
********************
start to train all models. 2025-05-29 12:50:11.753648
14's training. status=start 2025-05-29 12:50:11.753664
14's training. status=end 2025-05-29 12:50:59.310317
24's training. status=start 2025-05-29 12:50:59.310412
24's training. status=end 2025-05-29 12:51:26.956063
completed to train all models. 2025-05-29 12:51:26.956161
train to create model(53). 2025-05-29 12:51:29.956253
CPU 시간: 99.559979 초
메모리 사용량: 1029376 KB
********************
working model_version = 53  2025-05-29 12:51:57.840054
********************
start to train all models. 2025-05-29 12:51:57.840081
14's training. status=start 2025-05-29 12:51:57.840092
14's training. status=end 2025-05-29 12:52:47.200209
24's training. status=start 2025-05-29 12:52:47.200307
24's training. status=end 2025-05-29 12:53:14.434942
completed to train all models. 2025-05-29 12:53:14.435042


In [12]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()